In [3]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)

 
# splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
# print(splicing_feature_data.shape)
# splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [4]:
import pybedtools

In [5]:
alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive = pybedtools.BedTool(constitutive_bedfile)

In [6]:
from Bio import SeqIO
import kvector
import pybedtools
import pyhomer

DIRECTIONS = 'upstream', 'downstream'

placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
conservation_bed = pybedtools.BedTool(placental_filename)

genome = 'hg19'

nt = 200

genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

pair = pyhomer.ForegroundBackgroundPair(alt_exons, constitutive)
# #         print '\n', pair 

# for direction in DIRECTIONS:
#     print '\n\t', direction
#     intron_pair = pair.flanking_intron(direction, 'hg19', nt)

#     conserved_introns = intron_pair.intersect(conservation_bed, 'placental')
# #             print '\n', conserved_introns
# #     break
#     %time seqs = conserved_introns.foreground.sequence(fi=genome_fasta, s=True)
#     break
#     %time foreground_kmers = kvector.count_kmers(seqs.seqfn)

#     %time seqs = conserved_introns.background.sequence(fi=genome_fasta, s=True)
#     %time background_kmers = kvector.count_kmers(seqs.seqfn)
    
    
# #     kmer_zscore = (foreground_kmers.mean() - background_kmers.mean())/background_kmers.std()
#     kmer_zscore.name = '{}{}nt_{}_{}_placental'.format(direction, nt, phenotype, modality)
#     kmer_zscores.append(kmer_zscore)
# kmer_zscores_all = pd.concat(kmer_zscores, axis=1)
# kmer_zscores_all.head()

In [7]:
pyhomer.pyhomer.get_flanking_intron??

In [8]:
pair.flanking_intron??

In [9]:
import six

In [10]:
# %%time

# genome = 'hg19'

# kmers = {'upstream': [], 'downstream': []}

# for interval in alt_exons:
#     bed = pybedtools.BedTool([interval])
#     for direction in DIRECTIONS:
# #         print '\n\t', direction
#         if direction == 'downstream':
#             intron = bed.flank(l=0, r=nt, s=True, genome=genome)
#         elif direction == 'upstream':
#             intron = bed.flank(l=nt, r=0, s=True, genome=genome)
        
#         conserved_intron = intron.intersect(conservation_bed)
#         seqs = conserved_intron.sequence(fi=genome_fasta, s=True)
#         k = kvector.count_kmers(seqs.seqfn).sum()
# #         six.print_(k[k > 0])
#         k.name = interval.name
#         kmers[direction].append(k)
# #     break

In [11]:
# for direction, counts in kmers.items():
#     df = pd.concat([counts])
#     six.print_(df.head())
#     df.to_csv('{}/kmers_{}{}nt_placental.csv'.format(alternative_feature_folder, direction, nt)

In [12]:
range(3)

[0, 1, 2]

In [14]:
def count_intron_kmers(bed, genome, nt, genome_fasta, folder, suffix='', intersect=None):
    genome = 'hg19'

    kmers = {'upstream': [], 'downstream': []}

    for interval in bed:
        minibed = pybedtools.BedTool([interval])
        for direction in DIRECTIONS:
    #         print '\n\t', direction
            if direction == 'downstream':
                intron = minibed.flank(l=0, r=nt, s=True, genome=genome)
            elif direction == 'upstream':
                intron = minibed.flank(l=nt, r=0, s=True, genome=genome)

            if intersect is not None:
                intron = intron.intersect(intersect)
            seqs = intron.sequence(fi=genome_fasta, s=True)
            k = kvector.count_kmers(seqs.seqfn).sum()
    #         six.print_(k[k > 0])
            k.name = interval.name
            kmers[direction].append(k)
    for direction, counts in kmers.items():
        df = pd.concat([counts])
        six.print_(df.head())
        df.to_csv('{}/kmers_{}{}nt{}.csv'.format(folder, direction, nt, suffix))

## Try to parallelize

In [2]:
DNA = "ACGT"

def per_interval_kmers(bed, genome_fasta, intersect=None,
                       kmer_lengths=(4, 5, 6), residues=DNA):
    """Create a matrix of k-mer observations for each genomic region

    Parameters
    ----------
    bed : str or pybedtools.BedTool
        Either a filepath or pybedtools.BedTool of the genomic intervals whose
        kmers you want to count
    genome_fasta : str
        Path to the genome fasta file
    intersect : str or pybedtools.BedTool
        Either a filepath or pybedtools.BedTool of another region location,
        e.g. conserved elements, that you want to intersect with when
        searching for k-mers

    Returns
    -------
    kmers : pandas.DataFrame
        A (n_intervals, n_kmers) matrix of the number of DNA words observed in
        each interval, possibly filtered on only the regions that intersect
        with the original intervals
    """
    if not isinstance(bed, pybedtools.BedTool):
        bed = pybedtools.BedTool(bed)

    if intersect is not None and not isinstance(intersect, pybedtools.BedTool):
        intersect = pybedtools.BedTool(intersect)

    kmers = []

    for interval in bed:
        minibed = pybedtools.BedTool([interval])
        if intersect is not None:
            minibed = minibed.intersect(intersect)
        seqs = minibed.sequence(fi=genome_fasta, s=True)
        k = count_kmers(seqs.seqfn, kmer_lengths=kmer_lengths,
                        residues=residues).sum()
        k.name = interval.name
        kmers.append(k)
    return pd.concat(kmers, axis=1).T

In [31]:
! head -n 100 $alt_exons_bedfile > $alternative_feature_folder/test.bed

In [32]:
bed = pybedtools.BedTool('{}/test.bed'.format(alternative_feature_folder))

In [33]:
%%time
intersect = None
kmer_lengths = 4, 5, 6
residues = DNA

counts = []

for interval in bed:
    minibed = pybedtools.BedTool([interval])
    if intersect is not None:
        minibed = minibed.intersect(intersect)
    seqs = minibed.sequence(fi=genome_fasta, s=True)
    k = kvector.count_kmers(seqs.seqfn, kmer_lengths=kmer_lengths,
                    residues=residues).sum()
    k.name = interval.name
    counts.append(k)
true_kmers = pd.concat(counts, axis=1).T

CPU times: user 23.7 s, sys: 449 ms, total: 24.1 s
Wall time: 25.2 s


In [34]:
intersect = None
kmer_lengths = 4, 5, 6
residues = DNA

counts = []

def _count_kmers_single_interval(interval, genome_fasta, intersect, kmer_lengths, residues):
    minibed = pybedtools.BedTool([interval])
    if intersect is not None:
        minibed = minibed.intersect(intersect)
    seqs = minibed.sequence(fi=genome_fasta, s=True)
    k = kvector.count_kmers(seqs.seqfn, kmer_lengths=kmer_lengths,
                    residues=residues).sum()
    k.name = interval.name
    return k


In [35]:
import joblib

In [36]:
%%time
counts = joblib.Parallel(n_jobs=4)(
    joblib.delayed(_count_kmers_single_interval)(interval, genome_fasta, intersect, kmer_lengths, residues) for interval in bed)

CPU times: user 358 ms, sys: 79 ms, total: 437 ms
Wall time: 6.89 s


In [37]:
test_kmers = pd.concat(counts, axis=1).T

In [38]:
import pandas.util.testing as pdt

In [39]:
pdt.assert_frame_equal(test_kmers, true_kmers)

## Try with an intersection

In [40]:
intersect = conservation_bed

In [41]:
%%time
counts = joblib.Parallel(n_jobs=4)(
    joblib.delayed(_count_kmers_single_interval)(interval, genome_fasta, intersect, kmer_lengths, residues) for interval in bed)

CPU times: user 556 ms, sys: 150 ms, total: 706 ms
Wall time: 3min 20s


In [43]:
test_kmers = pd.concat(counts, axis=1).T

In [42]:
%%time
intersect = None
kmer_lengths = 4, 5, 6
residues = DNA

counts = []

for interval in bed:
    minibed = pybedtools.BedTool([interval])
    if intersect is not None:
        minibed = minibed.intersect(intersect)
    seqs = minibed.sequence(fi=genome_fasta, s=True)
    k = kvector.count_kmers(seqs.seqfn, kmer_lengths=kmer_lengths,
                    residues=residues).sum()
    k.name = interval.name
    counts.append(k)
true_kmers = pd.concat(counts, axis=1).T

CPU times: user 23.8 s, sys: 514 ms, total: 24.3 s
Wall time: 25.4 s


In [44]:
pdt.assert_frame_equal(test_kmers, true_kmers)